In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from component.model import BenefitModel
from component.model import ConstraintModel
from component.model import CostModel
from component.model import SeplanAoi

from component.scripts.seplan import Seplan


from sepal_ui.mapping import SepalMap, InspectorControl
from component.scripts.seplan import _percentile

In [37]:
seplan_aoi = SeplanAoi(aoi_model=None)
benefit_model = BenefitModel()
constraint_model = ConstraintModel()
cost_model = CostModel()
seplan_model = Seplan(
    seplan_aoi,
    benefit_model,
    constraint_model,
    cost_model,
)

In [38]:
import ee

ee.Initialize()

In [39]:
def _quintile(ee_image, ee_aoi, scale=100, name="layer_id"):
    """use quintile normailzation"""

    # test if the quintile list is valid
    # if the all area benefits is masked then this list will be empty and the image should be replaced by 1 everywhere
    # it can happen as the carbon sequestration layer has holes in south america and africa
    if valid_quintiles.size().getInfo() == 0:
        raise ValueError("cm.compute.error.missing_priority.format(name)")

    vaild_quintiles_list = valid_quintiles.toList(valid_quintiles.size())

    def conditions(feature):
        feature = ee.Feature(feature)

        quintiles = ee.Image().byte()
        quintiles = quintiles.paint(ee.FeatureCollection(feature), 0)

        low = ee.Number(feature.get("low"))
        lowmed = ee.Number(feature.get("lowmed"))
        highmed = ee.Number(feature.get("highmed"))
        high = ee.Number(feature.get("high"))

        out = (
            quintiles.where(ee_image.lte(low), 1)
            .where(ee_image.gt(low).And(ee_image.lte(lowmed)), 2)
            .where(ee_image.gt(lowmed).And(ee_image.lte(highmed)), 3)
            .where(ee_image.gt(highmed).And(ee_image.lte(high)), 4)
            .where(ee_image.gt(high), 5)
        )

        return out

    return ee.ImageCollection(vaild_quintiles_list.map(conditions)).mosaic()

In [41]:
# Caluclate quintiles for all default assets on benefits

benefits = [ee.Image(benefit_id) for benefit_id in seplan_model.benefit_model.assets]

## Define variables


In [42]:
ee_image = ee.Image(benefits[0])
ee_aoi = seplan_aoi.feature_collection
scale = ee_image.projection().nominalScale().multiply(2)

## Test with reduceRegion


In [43]:
band_name = ee.String(ee_image.bandNames().get(0))
quintiles_dict = ee_image.reduceRegion(
    reducer=ee.Reducer.percentile(percentiles=[20, 40, 60, 80]),
    geometry=ee_aoi,
    tileScale=2,
    scale=scale,
    maxPixels=1e13,
)

In [44]:
quintiles_names = ee.List(["p20", "p40", "p60", "p80"]).map(
    lambda quintile_name: band_name.cat("_").cat(ee.String(quintile_name))
)

In [46]:
quintiles_names

In [45]:
import json

json.loads(quintiles_names)

TypeError: the JSON object must be str, bytes or bytearray, not List

### Reclassify quintiles


In [32]:
low = ee.Number(quintiles_dict.get(quintiles_names.get(0)))
lowmed = ee.Number(quintiles_dict.get(quintiles_names.get(1)))
highmed = ee.Number(quintiles_dict.get(quintiles_names.get(2)))
high = ee.Number(quintiles_dict.get(quintiles_names.get(3)))

In [33]:
quintiles = (
    ee.Image(0)
    .where(ee_image.lte(low), 1)
    .where(ee_image.gt(low).And(ee_image.lte(lowmed)), 2)
    .where(ee_image.gt(lowmed).And(ee_image.lte(highmed)), 3)
    .where(ee_image.gt(highmed).And(ee_image.lte(high)), 4)
    .where(ee_image.gt(high), 5)
)

In [34]:
quintiles.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 5},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

## Test with reduceRegionS


In [8]:
from sepal_ui.mapping import SepalMap

map_ = SepalMap()

In [11]:
map_.add_ee_layer(ee_aoi)
map_.centerObject(ee_aoi)
map_

SepalMap(bottom=8519.0, center=[-2.7943488256019813, 58.29159111795809], controls=(ZoomControl(options=['posit…

In [12]:
# get the number of features of the ee_aoi

ee_aoi.size().getInfo()

1

In [13]:
quintile_collection = ee_image.reduceRegions(
    collection=ee_aoi,
    reducer=ee.Reducer.percentile(
        percentiles=[20, 40, 60, 80],
        outputNames=["low", "lowmed", "highmed", "high"],
    ),
    tileScale=2,
    scale=scale,
)
labels = ["high", "low", "lowmed", "highmed"]
# only use features that have non null quintiles
valid_quintiles = quintile_collection.filter(ee.Filter.notNull(labels))

In [14]:
quintile_values = {}
for f_id, feature in enumerate(valid_quintiles.getInfo()["features"]):
    quintile_values[f_id] = {}.setdefault(f_id, {})
    for prop, val in feature["properties"].items():
        if prop in labels:
            quintile_values[f_id][prop] = val

In [15]:
quintile_values

{0: {'high': 0.9585266642062268,
  'highmed': 0.9514316618442535,
  'low': 0.9237051010131836,
  'lowmed': 0.9410230284915759}}